In [39]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# from bioinfokit.analys import stat
import researchpy as rp
import numpy as np
import datetime
import fiona
from shapely.geometry import Point, Polygon, MultiPolygon

In [40]:
def read_data(path):
    return pd.read_csv(path)

In [41]:
file_path = "./datasets/Motor_Vehicle_Collisions_-_Person.csv"
data = read_data(file_path)

### Shape of dataset

In [42]:
data.shape

(4212666, 21)

In [43]:
data['CRASH_DATE'] = pd.to_datetime(data['CRASH_DATE'])
df = data.loc[(data['CRASH_DATE'] >= datetime.datetime(2014, 9, 13)) & (data['CRASH_DATE'] <= datetime.datetime(2019, 11, 24))] 

In [44]:
df.shape

(3549975, 21)

In [45]:
df.dtypes

UNIQUE_ID                         int64
COLLISION_ID                      int64
CRASH_DATE               datetime64[ns]
CRASH_TIME                       object
PERSON_ID                        object
PERSON_TYPE                      object
PERSON_INJURY                    object
VEHICLE_ID                      float64
PERSON_AGE                       object
EJECTION                         object
EMOTIONAL_STATUS                 object
BODILY_INJURY                    object
POSITION_IN_VEHICLE              object
SAFETY_EQUIPMENT                 object
PED_LOCATION                     object
PED_ACTION                       object
COMPLAINT                        object
PED_ROLE                         object
CONTRIBUTING_FACTOR_1            object
CONTRIBUTING_FACTOR_2            object
PERSON_SEX                       object
dtype: object

In [46]:
pd.set_option('display.max_columns', None)
df.head()

,UNIQUE_ID,COLLISION_ID,CRASH_DATE,CRASH_TIME,PERSON_ID,PERSON_TYPE,PERSON_INJURY,VEHICLE_ID,PERSON_AGE,EJECTION,EMOTIONAL_STATUS,BODILY_INJURY,POSITION_IN_VEHICLE,SAFETY_EQUIPMENT,PED_LOCATION,PED_ACTION,COMPLAINT,PED_ROLE,CONTRIBUTING_FACTOR_1,CONTRIBUTING_FACTOR_2,PERSON_SEX
0,10249006,4229554,2019-10-26,9:43,31aa2bc0-f545-444f-8cdb-f1cb5cf00b89,Occupant,Unspecified,19141108.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Registrant,NaN,NaN,U
1,10255054,4230587,2019-10-25,15:15,4629e500-a73e-48dc-b8fb-53124d124b80,Occupant,Unspecified,19144075.0,33,Not Ejected,Does Not Apply,Does Not Apply,"Front passenger, if two or more persons, inclu...",Lap Belt & Harness,NaN,NaN,Does Not Apply,Passenger,NaN,NaN,F
2,10253177,4230550,2019-10-26,17:55,ae48c136-1383-45db-83f4-2a5eecfb7cff,Occupant,Unspecified,19143133.0,55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Registrant,NaN,NaN,M
3,6650180,3565527,2016-11-21,13:05,2782525,Occupant,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Notified Person,NaN,NaN,NaN
4,10255516,4231168,2019-10-25,11:16,e038e18f-40fb-4471-99cf-345eae36e064,Occupant,Unspecified,19144329.0,7,Not Ejected,Does Not Apply,Does Not Apply,Right rear passenger or motorcycle sidecar pas...,Lap Belt,NaN,NaN,Does Not Apply,Passenger,NaN,NaN,F


### Columns with null values

In [47]:
def cols_with_null(df):
    sum_nan = df.isnull().sum().to_frame()
    sum_nan.columns=['#NaN']
    sum_nan_thresh = sum_nan[sum_nan['#NaN'] > 0]
    pd.set_option('display.max_rows', None)
    print(sum_nan_thresh)

In [48]:
cols_with_null(df)

                          #NaN
PERSON_ID                   19
VEHICLE_ID              119981
PERSON_AGE              245707
EJECTION               1678772
EMOTIONAL_STATUS       1636417
BODILY_INJURY          1636372
POSITION_IN_VEHICLE    1678514
SAFETY_EQUIPMENT       1678569
PED_LOCATION           3504494
PED_ACTION             3504587
COMPLAINT              1636367
PED_ROLE                 76315
CONTRIBUTING_FACTOR_1  3505700
CONTRIBUTING_FACTOR_2  3505760
PERSON_SEX              331832


In [49]:
df = df.drop(['PED_LOCATION', 'PED_ACTION', 'CONTRIBUTING_FACTOR_1', 'CONTRIBUTING_FACTOR_2'], axis=1)

In [50]:
cols_with_null(df)

                        #NaN
PERSON_ID                 19
VEHICLE_ID            119981
PERSON_AGE            245707
EJECTION             1678772
EMOTIONAL_STATUS     1636417
BODILY_INJURY        1636372
POSITION_IN_VEHICLE  1678514
SAFETY_EQUIPMENT     1678569
COMPLAINT            1636367
PED_ROLE               76315
PERSON_SEX            331832


In [51]:
def rows_nan(df):
    sum_nan_by_row = df.isnull().sum(axis=1).to_frame()
    sum_nan_by_row.columns=['#NaN']
    # sum_nan_thresh = sum_nan[sum_nan['#NaN'] > 0]
    pd.set_option('display.max_rows', 30)
    print(sum_nan_by_row)
    return sum_nan_by_row

In [52]:
temp = rows_nan(df)

         #NaN
0           7
1           0
2           6
3           9
4           0
...       ...
4210688     0
4210742     0
4210754     0
4210791     6
4210795     7

[3549975 rows x 1 columns]


In [53]:
np.max(temp)

#NaN    10
dtype: int64

### Number of unique values by column

In [54]:
df_unique = df.nunique().to_frame()
df_unique.columns=['#Unique']
df_unique

,#Unique
UNIQUE_ID,3549975
COLLISION_ID,897562
CRASH_DATE,1899
CRASH_TIME,1440
PERSON_ID,3473714
PERSON_TYPE,3
PERSON_INJURY,3
VEHICLE_ID,1597756
PERSON_AGE,797
EJECTION,6


In [55]:
index_names = df[(pd.isna(df["PERSON_AGE"]))].index
df_age = df.drop(index_names) 
index_names = df_age.loc[df['PERSON_AGE'].map(str).apply(len) >= 4].index
df_age = df_age.drop(index_names) 

In [56]:
df.shape

(3549975, 17)

In [57]:
df_age.shape

(3302041, 17)

In [58]:
df_unique = df_age.nunique().to_frame()
df_unique.columns=['#Unique']
df_unique.loc['PERSON_AGE']

#Unique    367
Name: PERSON_AGE, dtype: int64

In [60]:
df_age['PERSON_AGE'] = df_age['PERSON_AGE'].astype(int) 

In [62]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
rp.summary_cat(df_age["PERSON_AGE"])


,Variable,Outcome,Count,Percent
0,PERSON_AGE,0,533682,16.16
1,,27,70293,2.13
2,,28,70260,2.13
3,,29,70150,2.12
4,,30,69500,2.10
5,,26,68348,2.07
6,,31,66525,2.01
7,,32,65611,1.99
8,,25,64955,1.97
9,,33,63291,1.92


In [63]:
pd.set_option('display.max_rows', 30)

In [64]:
df_age = df_age.loc[(df_age['PERSON_AGE'] > 0) & (df_age['PERSON_AGE'] <= 122)]
df_age.shape

(2767288, 17)

In [65]:
pd.set_option('display.max_rows', None)
rp.summary_cat(df_age["PERSON_AGE"])

,Variable,Outcome,Count,Percent
0,PERSON_AGE,27,70293,2.54
1,,28,70260,2.54
2,,29,70150,2.53
3,,30,69500,2.51
4,,26,68348,2.47
5,,31,66525,2.40
6,,32,65611,2.37
7,,25,64955,2.35
8,,33,63291,2.29
9,,34,61736,2.23


In [67]:
cols_with_null(df_age)

                       #NaN
PERSON_ID                13
VEHICLE_ID            61696
EJECTION             942187
EMOTIONAL_STATUS     901250
BODILY_INJURY        901204
POSITION_IN_VEHICLE  941955
SAFETY_EQUIPMENT     942007
COMPLAINT            901195
PED_ROLE              71258
PERSON_SEX            71695
